In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%%capture
import pandas as pd
import networkx as nx
pd.set_option('display.max_columns', None) # so it prints all the dataframe columns ...
!pip install metapub
citations = pd.read_csv("/content/drive/MyDrive/citations.csv")

In [ ]:
def get_empty(row):
  if ( row["references"] == "[]" ):
    return row["pmid"]
  return ""

empty_row_iter = citations.apply( get_empty, axis=1 )
empty_row_iter = [x for x in empty_row_iter if x != ""]
citations = citations[~citations["pmid"].isin(empty_row_iter) ] # here ~ is equivalent to NOT (negates the statement)
citations

,pmid,references
0,22815718,"[{""citation"": ""WHO. WHO Report 2011: Global tu..."
1,27286562,"[{""citation"": ""World Health Orgnazation. Globa..."
2,24167451,"[{""citation"": ""World Health Organization (2011..."
3,27366339,"[{""citation"": ""World Health Organisation. Key ..."
4,26311855,"[{""citation"": ""WHO. 2014. \nGlobal tuberculosi..."
...,...,...
6247,8727433,"[{""citation"": ""J Trauma. 1989 Aug;29(8):1168-7..."
6248,19480694,"[{""citation"": ""Frerichs I, Dargaville PA, Dudy..."
6249,27075018,"[{""citation"": ""Ultraschall Med. 2012 Feb;33(1)..."
6250,22787182,"[{""citation"": ""Vardulaki KA, Prevost TC, Walke..."


In [ ]:
import json
data = {"pmid": [], "citation": []}
citation_df = pd.DataFrame(data)

def parse_citations(row):
  references_json = json.loads(row["references"])
  for ref in references_json:
    if ( "pubmed" in ref["article_ids"] ):
      citation_df.loc[len(citation_df)] = [row["pmid"], "PMID-"+str(ref["article_ids"]["pubmed"])]
      continue
    if ( "pmc" in ref["article_ids"] ):
      citation_df.loc[len(citation_df)] = [row["pmid"], "PMC-"+str(ref["article_ids"]["pmc"])]
      continue

    citation_df.loc[len(citation_df)] = [row["pmid"], "TITLE-"+str(ref["citation"])]
    continue

citations.apply( parse_citations, axis=1 )
citation_df = citation_df.astype(str)
citation_df

,pmid,citation
0,22815718,TITLE-WHO. WHO Report 2011: Global tuberculosi...
1,22815718,PMID-19246021
2,22815718,PMID-7902049
3,22815718,PMID-22362849
4,22815718,PMID-21179183
...,...,...
196903,22004635,PMID-18312783
196904,22004635,PMID-10761940
196905,22004635,PMID-15908528
196906,22004635,PMID-15069134


In [ ]:
citation_df.describe()

,pmid,citation
count,196908,196908
unique,6252,124262
top,25268160,PMID-7814313
freq,394,377


In [ ]:
# Citation network with ALL citations ...

citation_network = nx.from_pandas_edgelist(citation_df, "pmid", "citation")
# nx.draw_spring(citation_network) # draw_planar / draw / draw_spring - those are the best visualisation functions ...
print( "Number of connected components: " + str( nx.number_connected_components(citation_network) ) )

Number of connected components: 403


In [ ]:
# CODE FOR MAKING A CITATION NETWORK FOR EACH ARTICLE INDIVIDUALLY

# initial_article_pmids = list(citation_df['pmid'].unique())

# # article_dfs=[]
# for initial_article_pmid in initial_article_pmids:
#   article_df = citation_df[citation_df["pmid"]==initial_article_pmid]

#   citation_network = nx.from_pandas_edgelist(article_df, "pmid", "citation")
#   nx.draw_spring(citation_network) # draw_planar / draw / draw_spring - those are the best visualisation functions ...
#   print( "Number of connected components: " + str( nx.number_connected_components(citation_network) ) )

#   # article_dfs.append( article_df )

# article_df